In [70]:
import pandas as pd
import numpy as np

# Load Data
file_path = 'tayara_v3.csv' 
df = pd.read_csv(file_path)

# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

df.loc[(df['Nature'].isnull()) & (df['Prix'] > 100000), 'Nature'] = 'À Vendre'

# Step 4: Check missing values after the update (optional, for verification)
print("Missing values after:", df['Nature'].isnull().sum())

Missing values per column:
Région                     0
Localisation            2070
Type de bien            1319
Nature                   105
Superficie               335
Nb_Salles De bain        315
Nb_Chambres              221
Prix                      15
Date_De_Modification    2240
Texte Annonce              0
dtype: int64
Missing values after: 31


In [71]:
# Step 2: Check missing values for 'Type de bien' (optional, for verification)
print("Missing values before:", df['Type de bien'].isnull().sum())

# Convert 'Nb_Chambres' to numeric, forcing errors to NaN
df['Nb_Chambres'] = pd.to_numeric(df['Nb_Chambres'], errors='coerce')

# Step 3: Fill missing values in 'Type de bien' based on the given rules
df.loc[(df['Type de bien'].isnull()) & 
       (df['Nb_Chambres'].between(1, 3)) & 
       (df['Nb_Salles De bain'].between(1, 2)), 'Type de bien'] = 'Appartement'

df.loc[(df['Type de bien'].isnull()) & 
       (df['Nb_Chambres'].between(4, 5)) & 
       (df['Nb_Salles De bain'].between(1, 2)), 'Type de bien'] = 'Maison'

df.loc[(df['Type de bien'].isnull()) & 
    (df['Nb_Chambres'] > 5) & 
    (df['Nb_Salles De bain'] >= 2), 'Type de bien'] = 'Villa'

# Step 4: Check missing values after the update (optional, for verification)
print("Missing values after:", df['Type de bien'].isnull().sum())

#drop the rows that has no 'Type de bien' value
df = df.dropna(subset=['Type de bien'])

# Step 4: Check missing values after the update (optional, for verification)
print("Missing values after:", df['Type de bien'].isnull().sum())


Missing values before: 1319
Missing values after: 313
Missing values after: 0


In [72]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression

# 1️⃣ Load Data
file_path = 'tayara_v3.csv' 
df = pd.read_csv(file_path)

# 2️⃣ Drop 'Date_De_Modification' if it exists
df.drop(columns=['Date_De_Modification'], inplace=True, errors='ignore')

# 3️⃣ Handle Missing Values for 'Localisation' using KNN
df['Localisation_num'] = df['Localisation'].astype('category').cat.codes
localisation_imputer = KNNImputer(n_neighbors=5)
df['Localisation_num'] = localisation_imputer.fit_transform(df[['Localisation_num']])

localisation_mapping = dict(enumerate(df['Localisation'].astype('category').cat.categories))
df['Localisation'] = df['Localisation_num'].round().astype(int).map(localisation_mapping)
df.drop(columns=['Localisation_num'], inplace=True)

# 4️⃣ Handle Missing Values for 'Superficie' by filling based on 'Type de bien'
df['Superficie'] = df.groupby('Type de bien')['Superficie'].transform(lambda x: x.fillna(x.median()))

# 5️⃣ Handle Missing Values for 'Nb_Chambres' and make them integer
df['Nb_Chambres'] = pd.to_numeric(df['Nb_Chambres'], errors='coerce')  # Ensure numeric values
knn_imputer = KNNImputer(n_neighbors=5)
df[['Nb_Chambres']] = knn_imputer.fit_transform(df[['Nb_Chambres']])
df['Nb_Chambres'] = df['Nb_Chambres'].round()

# 6️⃣ Handle Missing Values for 'Nb_Salles De bain'
df['Nb_Salles De bain'] = df.groupby(['Type de bien', 'Localisation'])['Nb_Salles De bain'].transform(
    lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x.fillna(1)
)
df['Nb_Salles De bain'] = df['Nb_Salles De bain'].round()

# 7️⃣ Impute rows where 'Nb_Chambres' == 0 (use median or KNN)
df['Nb_Chambres'] = df['Nb_Chambres'].replace(0, np.nan)  # Convert 0 to NaN for imputation
df['Nb_Chambres'] = df.groupby('Type de bien')['Nb_Chambres'].transform(lambda x: x.fillna(x.median()))

# 8️⃣ Impute rows where 'Nb_Salles De bain' == 0 (use mode or KNN)
df['Nb_Salles De bain'] = df['Nb_Salles De bain'].replace(0, np.nan)  # Convert 0 to NaN for imputation
df['Nb_Salles De bain'] = df.groupby('Type de bien')['Nb_Salles De bain'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else 1))

# 9️⃣ Drop rows where 'Type de bien' or 'Localisation' are missing
df.dropna(subset=['Type de bien', 'Localisation'], inplace=True)

# 🔟 Filter 'Prix' to handle unrealistic values
df.loc[(df['Prix'] < 50000) | (df['Prix'] > 1300000), 'Prix'] = 0

# 🔍 Check for NaNs in key columns
print("Missing values after cleaning:")
print(df[['Superficie', 'Nb_Chambres', 'Nb_Salles De bain', 'Type de bien', 'Localisation']].isnull().sum())

# 1️⃣1️⃣ Prepare data for regression (use only rows where 'Prix' is not NaN or 0)
X = df.loc[(df['Prix'].notnull()) & (df['Prix'] != 0), ['Superficie', 'Nb_Chambres', 'Nb_Salles De bain', 'Type de bien', 'Localisation']]
y = df.loc[(df['Prix'].notnull()) & (df['Prix'] != 0), 'Prix']

# 1️⃣2️⃣ Convert categorical columns ('Type de bien', 'Localisation') to dummy variables
X = pd.get_dummies(X, columns=['Type de bien', 'Localisation'], drop_first=True)

# 1️⃣3️⃣ Train the regression model
regressor = LinearRegression()
regressor.fit(X, y)

# 1️⃣4️⃣ Prepare X for missing 'Prix' (where 'Prix' is NaN or 0)
X_missing_prix = df.loc[df['Prix'].isnull() | (df['Prix'] == 0), ['Superficie', 'Nb_Chambres', 'Nb_Salles De bain', 'Type de bien', 'Localisation']]

# Convert categorical features to dummies to match training data
X_missing_prix = pd.get_dummies(X_missing_prix, columns=['Type de bien', 'Localisation'], drop_first=True)

# Align columns to match the training data
X_missing_prix = X_missing_prix.reindex(columns=X.columns, fill_value=0)

# Fill missing numerical values in X_missing_prix
X_missing_prix.fillna({
    'Superficie': df['Superficie'].median(), 
    'Nb_Chambres': df['Nb_Chambres'].median(), 
    'Nb_Salles De bain': df['Nb_Salles De bain'].mode().iloc[0]
}, inplace=True)

# 1️⃣5️⃣ Predict missing 'Prix' using the trained model
predicted_prix = regressor.predict(X_missing_prix)

# 1️⃣6️⃣ Update missing 'Prix' (NaN and 0) with predicted values
df.loc[df['Prix'].isnull() | (df['Prix'] == 0), 'Prix'] = predicted_prix

# see the number of duplicates and drop the duplicates as well as the collumn text annonce and the modification date 
print("Number of duplicates before:", df.duplicated().sum())
df.drop(columns=['Texte Annonce'], inplace=True)
df.drop_duplicates(inplace=True)
print("Number of duplicates after:", df.duplicated().sum())

# 🔥 Final Cleaned Data
df.reset_index(drop=True, inplace=True)

# 🗂️ Save cleaned data to CSV
df.to_csv('cleaned_tayara.csv', index=False)

print("✅ Data cleaning and price prediction complete.")


Missing values after cleaning:
Superficie           0
Nb_Chambres          0
Nb_Salles De bain    0
Type de bien         0
Localisation         0
dtype: int64
Number of duplicates before: 42
Number of duplicates after: 0
✅ Data cleaning and price prediction complete.
